In [1]:
from __future__ import unicode_literals
from sklearn.externals import joblib
from textacy import fileio, preprocess, Doc, vsm
import numpy as np

In [2]:
#classifier
#msg_model = joblib.load('/home/pretrain-model/category-ann.pkl')
#id2term_itr = fileio.read_json('/home/pretrain-model/category-id2term-ann')
#label2id_itr = fileio.read_json('/home/pretrain-model/category-label2id-ann')

#regression
msg_model = joblib.load('/home/pretrain-model/category-ann-reg.pkl')
id2term_itr = fileio.read_json('/home/pretrain-model/category-id2term-ann-reg')
label2id_itr = fileio.read_json('/home/pretrain-model/category-label2id-ann-reg')

id2term = [i for i in id2term_itr][0]
label2id = [i for i in label2id_itr][0]

In [3]:
#print(label2id)

In [4]:
#sample_msg = u'Referring to the letters, "Circle Line train incident: SMRT explains cause and steps taken" and "Commuters distressed and kept in the dark", forum writer Larry Chong Tuck Lai asked several questions regarding the safety of passengers when there is smoke inside the train. These are the questions, assuming a fully packed peak-hour train: How much smoke is needed before the smoke detector is activated and the ventilation system shut? What happens after the ventilation system is shut? How long will the air supply last in a crowded train with no air-conditioning or ventilation and with the presence of smoke in the cabin? While the incident was resolved within five minutes, what happens if we have a case where staff are unable to reach the site of the breakdown for hours, and the system is not restored? What options are available to passengers trapped in the train? Are there manual systems to allow passengers to free themselves, or at least open windows to allow air to flow in?CIC/CMC is working with SMRT Trains to prepare a response.'
#sample_msg = u'Taxi operators are taking the fight to Uber and Grab by setting up their own private-hire and limousine arms. The aim is to retain drivers who have begun to defect to the fast-expanding private-hire fleets run by the third-party taxi app providers. SMRT Corp has ordered a fleet of more than 300 new Toyota Vellfire MPVs, Corolla Altis sedans and some Lexus cars for that purpose. Taxi industry players noted that unlike cab companies, the newcomers are not required to meet service standards, their fleet expansion will not be restricted, they do not need to buy insurance policies that cover passengers, there is no restriction on vehicle age, and their cars attract far lower road taxes than cabs. Others said the fast-expanding third-party fleet will eventually impact cabbies\' earnings as they are competing for a fixed market that is not growing nearly as fast.'
#sample_msg = u'Several bus services plying Orchard Road will be diverted temporarily on the first Saturday of the month for the next four months, due to a stretch of the road being closed for Pedestrian Night. The temporary route diversion of SMRT bus services 77, 106, 167, 171, 190, 700A and 972 will last between 6pm and 11pm each time.'
sample_msg = u'These were the evening news coverage of the BPLRT disruption on Monday. Mar 30.There was a train disruption along Bukit Panjang LRT again on Mon at around 6.45pm, the second time in three weeks, and passengers had to walk along the tracks again. Wanbao reported that this was the 10th train disruption in the last few months.Some passengers onboard the defective train commented that the train moved intermittently before coming to a halt. In-train announcements were barely audible and waited for 50 minutes before service staff detrained them to Bukit Panjang station.'

In [5]:
sample_doc = Doc(sample_msg)

In [6]:
terms_lists_test = sample_doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True)

In [7]:
dtm_test, i2t_test = vsm.doc_term_matrix([terms_lists_test], weighting='tf', normalize=True, max_n_terms=15000)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [8]:
dtm_test

<1x44 sparse matrix of type '<type 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

In [9]:
#build new index
test2train = dict()
for (i, j) in i2t_test.items():
    if j in id2term.values():
        train_i = id2term.keys()[id2term.values().index(j)]
        test2train[i]=train_i

In [10]:
test_feature = np.zeros((dtm_test.shape[0], len(id2term)))
dtm_test_array = dtm_test.toarray()

for (i, j) in test2train.items():
    test_feature[:, j] = dtm_test_array[:, i]

In [11]:
#classifier
#x = np.arange(len(label2id.keys()))[msg_model.predict(test_feature)[0] == 1]
#(x, label2id.items())

In [12]:
result = msg_model.predict(test_feature)
max_index = result.argmax()
msg_category = [i for (i, j) in label2id.items() if j == max_index]

In [15]:
msg_category

[u'smrt trains']

In [18]:
dict(zip(label2id.keys(), result[0]))

{u'competitors': 0.02247927901421265,
 u'forum': 0.021594028785345956,
 u'overseas news': -0.015910553127937939,
 u'public transport industry': -0.0077442201339843614,
 u'smrt buses': 0.1106549953452771,
 u'smrt corporate': 0.0089556842237016182,
 u'smrt taxis': 0.029400378017674367,
 u'smrt trains': 0.82689107029082709,
 u'taxi': -0.029409323485012923,
 u'unknown': 0.023589861395304397}